In [ ]:
import random
import numpy as np
import operator
import pandas as pd
from datetime import datetime, timedelta
import pickle
import time

In [ ]:
global total_utxo
global total_address
global pool
global b_review_index
global v_review_index
global SR_address
global SR_UTXO

In [ ]:
class UTXO(object):
    def __init__(self, value, address):
        self.value = value
        self.address = address

class Address(object):
    def __init__(self, id, utxo_set):
        self.id = id
        self.utxo_set = utxo_set
    def get_utxo(self, new_utxo):
        self.utxo_set.append(new_utxo)
        total_UTXO.append(new_utxo)
    def spend_utxo(self, old_utxo):
        self.utxo_set.remove(old_utxo)
        total_UTXO.remove(old_utxo)

In [ ]:
class Transaction(object):
    def __init__(self, id, type, sender, to, value):
        self.id = id
        self.type = type
        self.sender = sender
        self.to = to
        self.value =value

In [ ]:
dx = pd.read_excel('data.xlsx')
datar = []
for i in range(len(dx)):
    tmp = [dx["ID"][i], dx["Value"][i], dx["Date"][i]]
    ttmp = datetime.strptime(str(tmp[2]), "%Y%m%d")
    tmp[2] = ttmp
    datar.append(tmp)
    
#print(datar)
data = []
for v in datar:
    if v not in data:
        data.append(v)
data_t = []
for d in data:
    data_t.append(d[2])
start = min(data_t)
end = max(data_t)

r_data_index = np.random.choice(range(len(data)), 100, replace=False)
r_data = []
for r_index in r_data_index:
    data[r_index][1] = round(data[r_index][1]*pow(10,8))
    r_data.append(data[r_index])

b_review_index = []
v_review_index = []

for rd in r_data:
    time_difference = int((rd[2]-start).days)*5120
    br = random.choice(range(time_difference))
    while br in b_review_index:
        br = random.choice(range(time_difference))
    b_review_index.append(br)
    t_max = time_difference + 71680
    if t_max > 669999:
        t_max = 669999
    rv = random.choice(range(time_difference, t_max))
    while rv in v_review_index:
        rv = random.choice(range(time_difference, t_max))
    v_review_index.append(rv)
#print(len(b_review_index) == len(list(set(b_review_index))))
#print(len(v_review_index) == len(list(set(v_review_index))))
#print(len(b_review_index) == len(list(set(b_review_index) - set(v_review_index)))) 

In [ ]:
with open("co_value.pickle", "rb") as f:
    co_value = pickle.load(f)
co_value = list(co_value)

with open("rov.pickle", "rb") as f:
    relay_tx_output_value = pickle.load(f)
rov = list(relay_tx_output_value)        
        
with open("pic.pickle", "rb") as f:
    peel_tx_input_count = pickle.load(f)
peel_tx_input_count = list(peel_tx_input_count)

with open("sov.pickle", "rb") as f:
    sweep_tx_output_value = pickle.load(f)
sov = list(sweep_tx_output_value)

with open("siv.pickle", "rb") as f:
    sweep_tx_input_value = pickle.load(f)
siv = list(sweep_tx_input_value)

with open("ptx1ov.pickle", "rb") as f:
    ptx1ov = pickle.load(f)
ptx1ov = list(ptx1ov)

with open("ptx1iv.pickle", "rb") as f:
    ptx1iv = pickle.load(f)
ptx1iv = list(ptx1iv)

with open("ptxov.pickle", "rb") as f:
    ptxov = pickle.load(f)
ptxov = list(ptxov)

with open("ptxiv.pickle", "rb") as f:
    ptxiv = pickle.load(f)
ptxiv = list(ptxiv)

with open("dtxov.pickle", "rb") as f:
    dtxov = pickle.load(f)
dtxov = list(dtxov)

with open("mtxic.pickle", "rb") as f:
    mic = pickle.load(f)
mic = list(mic)

with open("mtxov.pickle", "rb") as f:
    mtxov = pickle.load(f)
mtxov = list(mtxov)

with open("mtxiv.pickle", "rb") as f:
    mtxiv = pickle.load(f)
mtxiv = list(mtxiv)

with open("utxo.pickle", "rb") as f:
    past_utxo = pickle.load(f)

In [ ]:
def make_ov(tt, b_flag, v_flag, review_value, DT):
    if tt == "relay" or tt == "sweep":
        if b_flag:
            ov = [review_value]
        else:
            if v_flag and tt =="relay":
                not_found = True
                for v in DT:
                    if v < review_value:
                        ov = [v]
                        not_found = False
                        break
                if not_found:
                    ov = [review_value]
            else:
                ov = [random.choice(DT)]
            
    else:
        if v_flag and (tt == "distribution" or tt == "peel1"):
            for vlist in DT:
                not_found = True
                if sum(vlist) < review_value:
                    ov = vlist
                    not_found = False
                    break
                if not_found:
                    ov = random.choice(DT)
        else:
            ov = random.choice(DT)
        
        if b_flag:
            ov = [review_value] + ov
    return ov

In [ ]:
def utxo_list_sum(utxo_list):
    utxo_sum = 0
    for u in utxo_list:
        utxo_sum += u.value
    return utxo_sum

In [ ]:
def find_utxos(tt, i, v_flag, review_value, ov_sum):
    spending_utxo = []
    
    if tt == "relay" or tt == "distribution" or tt == "peel1":
        if v_flag:
            sra = pool[b_review_index[v_review_index.index(i)]].to[0]
            spending_utxo.append(sra.utxo_set[0])
        else:
            not_found = True
            for u in total_UTXO:
                if (u.value > ov_sum) and (u not in SR_UTXO):
                    spending_utxo.append(u)
                    not_found = False
                    break
            if not_found:
                a = Address( len(total_Address) , [] )
                new_utxo = UTXO( int(ov_sum) , a )
                a.get_utxo(new_utxo)
                total_Address.append(a)
                spending_utxo.append(new_utxo)
                
    else:
        iv_sum = 0
        for j in range(2):
            su = random.choice(total_UTXO)
            while (su in SR_UTXO) or (su in spending_utxo):
                su = random.choice(total_UTXO)
            spending_utxo.append(su)
            iv_sum += su.value
            
        if iv_sum < ov_sum:
            a = Address( len(total_Address) , [] )
            new_utxo = UTXO( int(ov_sum-iv_sum) , a )
            a.get_utxo(new_utxo)
            total_Address.append(a)
            spending_utxo.append(new_utxo)
        
        if v_flag:
            sra = pool[b_review_index[v_review_index.index(i)]].to[0]
            spending_utxo = [sra.utxo_set[0]] + spending_utxo
            
    return spending_utxo 

In [ ]:
def make_to(tt, b_flag, oc, GNA_p):
    to = []
    for i in range(oc):
        if i == 0 and b_flag:
            a = Address( len(total_Address) , [] )
            total_Address.append(a)
            SR_address.append(a)
            to.append(a)
        else:
            GNA_bool = np.random.choice([True, False], p=GNA_p)
            if GNA_bool:
                a = Address( len(total_Address) , [] )
                total_Address.append(a)
                to.append(a)
            else:
                a = random.choice(total_Address)
                while a in SR_address:
                    a = random.choice(total_Address)
                to.append(a)
    return to

In [ ]:
tx_type = ["relay", "peel", "distribution", "sweep", "mimo"]
tx_type_p = [0.125, 0.7005, 0.07, 0.07, 0.0345]
start = time.time()
bool_list = [True, False]
pool = []
SR_address = []
SR_UTXO = []
total_Address = []
total_UTXO = []

past_utxo = list(past_utxo)
utxos = np.random.choice(past_utxo, 160000)
utxos = list(utxos)

for u in utxos:
    a = Address( len(total_Address) , [] )
    new_utxo = UTXO( int(u) , a )
    a.get_utxo(new_utxo)
    total_Address.append(a)

for i in range(670000):
    if i % 1000 == 0:
        print(i)
        print("time :", (time.time() - start)/60)
    #####################################################
    b_flag = False
    v_flag = False
    review_value = None
    if i in b_review_index:
        review_value = r_data[b_review_index.index(i)][1]
        b_flag = True
    elif i in v_review_index:
        review_value = r_data[v_review_index.index(i)][1]
        v_flag = True
    tt = np.random.choice(tx_type, p=tx_type_p)
    #####################################################   
    if tt == "relay":
        DT = rov
        GNA_p =[0.45, 0.55]
    elif tt == "peel":
        Input_one_bool = np.random.choice(bool_list, p=[0.77, 0.23])
        if Input_one_bool:
            tt = 'peel1'
            DT = ptx1ov
            GNA_p = [0.48, 0.52]
        else:
            DT = ptxov
            GNA_p = [0.45, 0.55]
    elif tt == "distribution":
        DT = dtxov
        GNA_p = [0.52,0.48]
    elif tt == "sweep":
        DT = sov
        GNA_p = [0.41, 0.59]
    else:
        DT = mtxov
        GNA_p = [0.41, 0.59] 
        
    ov = make_ov(tt, b_flag, v_flag, review_value, DT)
    spending_utxo = find_utxos(tt, i, v_flag, review_value, sum(ov))
    iv = []
    sender = []
    for su in spending_utxo:
        iv.append(su.value)
        sender.append(su.address)
        su.address.spend_utxo(su)
        
    to = make_to(tt, b_flag, len(ov), GNA_p)
    for j, v in enumerate(ov):
        a = to[j]
        new_utxo = UTXO( int(v) , a )
        a.get_utxo(new_utxo)
        if j == 0 and b_flag:
            SR_UTXO.append(new_utxo)
    value = [iv, ov]
    t = Transaction(i, tt, sender, to, value)
    pool.append(t)
    
print("time :", (time.time() - start)/60)

In [ ]:
matched_address = []
for rd in r_data:
    D = int((rd[2]-min(data_t)).days)*5120
    V = rd[1]
    otxca = []
    itxca = []
    ma = []
    
    for tx in pool[:D]:
        if V in tx.value[1]:
            otxca.append(tx.to[tx.value[1].index(V)])
            
    smax = D+14*5120
    if smax > len(pool):
        smax = len(pool)-1
    for tx in pool[D:smax]:
        if V in tx.value[0]:
            itxca.append(tx.sender[tx.value[0].index(V)])
    ca = list(set(otxca) & set(itxca))
    
    for a in ca:
        ic = 0
        oc = 0
        fail = False
        for tx in pool:
            if (ic > 1) or (oc > 1):
                fail = True
                break
            if a in tx.sender:
                ic += 1
            if a in tx.to:
                oc += 1
        if fail:
            continue
        if (ic == 1) and (oc == 1):
            ma.append(a)
            
    if len(ma) == 1:
        matched_address.append(ma[0])

In [ ]:
for bri in b_review_index:
    if pool[bri].to[0] not in matched_address:
        print(bri)

In [ ]:
FN_address = list(set(SR_address) - set(matched_address))

In [ ]:
for fna in FN_address:
    for tx in pool:
        if fna in tx.sender:
            print(tx.value[0][tx.sender.index(fna)])

In [ ]:
len(matched_address)

In [ ]:
len(SR_address)

In [ ]:
count = 0
for ma in matched_address:
    if ma in SR_address:
        count+=1
print(count)

In [ ]:
TP = count
FP = len(matched_address) - TP
FN = len(SR_address) - len(matched_address)

Recall = TP/(TP+FN)
Precision = TP/(TP+FP)
F1_Score = 2*(Recall * Precision)/(Recall + Precision)

print(F1_Score)

In [ ]:
TP = count
FP = len(matched_address) - TP
FN = len(SR_address) - len(matched_address)

Recall = TP/(TP+FN)
Precision = TP/(TP+FP)
print(Recall)
print(Precision)

In [ ]:
class Cluster(object):
    def __init__(self, id, address_set):
        self.id = id
        self.address_set = address_set
        
    def address_num(self):
        return len(self.address_set)    
    
    def cluster_sum(self, other_cluster):
        sas = set(self.address_set)
        ocas = set(other_cluster.address_set)
        if sas - ocas != sas:
            self.address_set = list(sas | ocas)
            clusters.remove(other_cluster)

In [ ]:
global clusters
clusters = []

for tx in pool:
    if tx.type == "sweep":
        c = Cluster(len(clusters), tx.sender)
        clusters.append(c)
        
    if tx.type == "peel1":
        if tx.value[1][0] > tx.value[1][1]:
            p_index = 1
        else:
            p_index = 0
        c = Cluster(len(clusters), tx.sender + [tx.to[p_index]])
        clusters.append(c)
        
for c in clusters:
    for i in range(clusters.index(c)+1, len(clusters)):
        if i >= len(clusters):
            break
        c.cluster_sum(clusters[i])

In [ ]:
SR_clusters = []
for ma in matched_address:
    for c in clusters:
        if ma in c.address_set:
            SR_clusters.append(c)
print(len(SR_clusters))

In [ ]:
len(total_Address)